# 基礎方程式 Orszag 1971 の (15)-(17) 式
$n$次チェビシェフ多項式$T_n$の$0\leq n\leq N-4$について, 
\begin{equation}
\begin{split}
&\dfrac{1}{24} \sum^{N}_{\substack{p=n+4\\ p\equiv n\; (\mathrm{mod} \; 2)}}{p\left[p^2(p^2-4)^2-3n^2p^4+3n^4p^2-n^2\left(n^2-4 \right) ^2-48\alpha ^2(p^2-n^2) \right] a_p} -8\alpha ^2(n+2)(n+1)a_{n+2} +\alpha ^4c_na_n \\
&\quad -\dfrac{i\alpha R}{2} \left[\sum^{N}_{p=2}{a_p\sum^{}_{\substack{m\equiv p\; (\mathrm{mod} \; 2) \\ |m|\leq p-2 \\ |n-m|\leq N}}{p(p^2-m^2)c_{|n-m|}b_{|n-m|} }} -\alpha ^2\sum^{}_{\substack{|p|\leq N \\ |n-p|\leq N}}{c_{|p|}a_{|p|}c_{|n-p|}b_{|n-p|}} -\sum^{}_{\substack{|p|\leq N \\ |n-p|\leq N}}{c_{|p|}a_{|p|}\sum^{N}_{\substack{m=|n-p|+2\\ m+n\equiv p\; (\mathrm{mod} \; 2)}}{m\left[m^2-(n-p)^2 \right] b_m}} \right] \\
&\quad +\lambda \left\{i\alpha R\sum^{N}_{\substack{p=n+2\\ p\equiv n\; (\mathrm{mod} \; 2)}}{p(p^2-n^2)a_p} -i\alpha ^3Rc_na_n \right\} =0.
\end{split}
 \label{eq:Orszag1971-17-A44}
\end{equation}
および境界条件から得られる係数の関係式:
\begin{align}
a_{N_{\mathrm{e}}}=&\; \sum^{N_{\mathrm{e}}-4}_{\substack{p=0\\ p=0\; (\mathrm{mod}\; 2)}}{\dfrac{(N_{\mathrm{e}}-2)^2-p^2}{4(N_{\mathrm{e}}-1)} a_p} ,\; (\mathrm{even mode}), \label{eq:Orszag1971-17-A45} \\
a_{N_{\mathrm{e}}-2}=&\; \sum^{N_{\mathrm{e}}-4}_{\substack{p=0\\ p=0\; (\mathrm{mod}\; 2)}}{\dfrac{p^2-N^2_{\mathrm{e}}}{4(N_{\mathrm{e}}-1)} a_p} ,\; (\mathrm{even mode}), \label{eq:Orszag1971-17-A46} \\
a_{N_{\mathrm{o}}}=&\; \sum^{N_{\mathrm{o}}-4}_{\substack{p=1\\ p=1\; (\mathrm{mod}\; 2)}}{\dfrac{(N_{\mathrm{o}}-2)^2-p^2}{4(N_{\mathrm{o}}-1)} a_p} ,\; (\mathrm{odd mode}), \label{eq:Orszag1971-17-A47} \\
a_{N_{\mathrm{o}}-2}=&\; \sum^{N_{\mathrm{o}}-4}_{\substack{p=1\\ p=1\; (\mathrm{mod}\; 2)}}{\dfrac{p^2-N^2_{\mathrm{o}}}{4(N_{\mathrm{o}}-1)} a_p} ,\; (\mathrm{odd mode}), \label{eq:Orszag1971-17-A48} 
\end{align}
求める固有値方程式は
$$
(A-B\lambda )\textbf{x} =\textbf{0} \quad \rightarrow \quad \left(B^{-1}A-\lambda I \right) \textbf{x} =\textbf{0},
$$
$$
\textbf{x} \equiv \left[a_0,a_1,\cdots ,a_N \right] ^T.
$$

In [6]:
# In this procedure, two methods are used to individually determine the eigenvalues:
# Method 1: [A-λB]x = 0 -> [B^{-1}A-λI]x = 0
#           here, to be regular matrix B, boundary conditions are contained in 0<=n<=N-4 (so, elements used in A/B are N-4xN-4.)
# Method 2: [A-λB]x = 0 -> Ax=λBx -> Schur decomposition
#           here, elements used in A2/B2 are NxN

#using .Lilly1966_solver
using OffsetArrays  # 配列要素番号を任意開始にするモジュール (行列計算時は 1 始まりの通常 matrix に copy する)
using LinearAlgebra

#-- Drawing parameters    
N = 56  # wavenumber in Chebyshev
Ncal = N + 1

# initialize working arrays
# A/B: using to determine the eigenvalue by Method 1,
# A2/B2: using to determine the eigenvalue by Method 2.
A_J = reshape(zeros(Complex{Float64},Ncal,Ncal),Ncal,Ncal)  # Matrix for calculation
B_J = reshape(zeros(Complex{Float64},Ncal,Ncal),Ncal,Ncal)  # Matrix for calculation
A2_J = reshape(zeros(Complex{Float64},Ncal,Ncal),Ncal,Ncal)  # Matrix for calculation
B2_J = reshape(zeros(Complex{Float64},Ncal,Ncal),Ncal,Ncal)  # Matrix for calculation
### --- From here, supposing matrix element as row, col as in mathematical notation
Acal = reshape(zeros(Complex{Float64},Ncal,Ncal),Ncal,Ncal)  # Matrix for calculation
Bcal = reshape(zeros(Complex{Float64},Ncal,Ncal),Ncal,Ncal)  # Matrix for calculation
A2cal = reshape(zeros(Complex{Float64},Ncal,Ncal),Ncal,Ncal)  # Matrix for calculation
B2cal = reshape(zeros(Complex{Float64},Ncal,Ncal),Ncal,Ncal)  # Matrix for calculation
A = OffsetArray(Acal,0:N,0:N)  # coefficient matrix
B = OffsetArray(Bcal,0:N,0:N)  # coefficient matrix
A2 = OffsetArray(Acal,0:N,0:N)  # coefficient matrix
B2 = OffsetArray(Bcal,0:N,0:N)  # coefficient matrix
### --- From here, supposing matrix element as col, row as in julia notation (only using these arrays for matrix calc)
b = OffsetArray(zeros(Float64,Ncal),0:N)  # A vector composed of Chebyshev coefficients for the basic state flow
c = OffsetArray(zeros(Float64,Ncal),0:N)  # factorized vector (c_0=2, c_n=1, (n>0))

# setting parameters
α = 1.0
Re = 10000.0

# setting the Chebyshev coefficients for the basic flow
b[0] = 0.5
#b[2] = -b[0]

# setting the factorized vector
c[0] = 2.0
c[1:N] .= 1.0

# setting maximum even and odd numbers from N
Nemax = 0
Nomax = 1
for i in 0:2:N
    Nemax = i
end
for i in 1:2:N
    Nomax = i
end
println("Nemax = ", Nemax, ", Nomax = ", Nomax)

# setting matrices
# 支配方程式から構築される部分 (チェビシェフ次数 0<=n<=N-4 の範囲)
# この段階では境界条件で成り立つ係数の関係は考慮しない
for n in 0:N-4
    n2 = n^2
    
    # a_n
    for p in n+4:2:N
        p2 = p^2
        A[n,p] = p * (p2 * (p2 - 4)^2 - 3 * n2 * p2 * (p2 - n2) - n2 * (n2 - 4)^2 - 48 * α^2 * (p2 - n2)) / 24.0
    end
    
    A[n,n+2] = - 8 * α^2 * (n + 2) * (n + 1)
    A[n,n] = α^4 * c[n]
    
    for p in 2:N
        acoef = 0.0
        p2 = p^2
        for m in -(p-2):2:(p-2)
            abs_n_m = abs(n - m)
            m2 = m^2
            if abs_n_m <= N
                acoef = acoef + p * (p2 - m2) * c[abs_n_m] * b[abs_n_m]
            end
        end
        A[n,p] = A[n,p] - complex(0.0,0.5 * α * Re * acoef)
    end

    for p in -N:N
        abs_n_p = abs(n - p)
        abs_p = abs(p)
        if abs_n_p <= N
            A[n,abs_p] = A[n,abs_p] + complex(0.0,0.5 * α^3 * Re * c[abs_p] * c[abs_n_p] * b[abs_n_p])
        end

        if abs_n_p+2 <= N
            acoef = 0.0
            for m in abs_n_p+2:2:N
                m2 = m^2
                acoef = acoef + m * (m2 - (n - p)^2) * b[m]
            end
            A[n,abs_p] = A[n,abs_p] + complex(0.0,0.5 * α * Re * c[abs_p] * acoef)
        end
    end

    # for lambda
    for p in n+2:2:N
        p2 = p^2
        B[n,p] = complex(0.0, - α * Re * p * (p2 - n2))
    end
    B[n,n] = complex(0.0, α^3 * Re * c[n])

end

# Copy A/B -> A2/B2
A2 .= A
B2 .= B

# For Method 1, 
## a_n (N-3<=n<=N) from boundary conditions
# A(B)[n,N-3:N] に含まれている値 x 境界条件から得られる係数分を A(B)[n,0:N-4] に足し合わせる
for n in 0:N-4  # For each coefficient of T_n (0<=n<=N-4)
    for p in 0:2:Nemax-4  # even mode
        #println(n,", ", p, ", ", A[n,p], ", ", (((Nemax - 2)^2 - p^2)/(4 * (Nemax - 1))) * A[n,Nemax], ",", ((p^2 - Nemax^2)/(4 * (Nemax - 1))) * A[n,Nemax-2])
        A[n,p] = A[n,p] + (((Nemax - 2)^2 - p^2)/(4 * (Nemax - 1))) * A[n,Nemax]
        A[n,p] = A[n,p] + ((p^2 - Nemax^2)/(4 * (Nemax - 1))) * A[n,Nemax-2]
        B[n,p] = B[n,p] + (((Nemax - 2)^2 - p^2)/(4 * (Nemax - 1))) * B[n,Nemax]
        B[n,p] = B[n,p] + ((p^2 - Nemax^2)/(4 * (Nemax - 1))) * B[n,Nemax-2]
    end
    for p in 1:2:Nomax-4  # odd mode
        A[n,p] = A[n,p] + (((Nomax - 2)^2 - p^2)/(4 * (Nomax - 1))) * A[n,Nomax]
        A[n,p] = A[n,p] + ((p^2 - Nomax^2)/(4 * (Nomax - 1))) * A[n,Nomax-2]
        B[n,p] = B[n,p] + (((Nomax - 2)^2 - p^2)/(4 * (Nomax - 1))) * B[n,Nomax]
        B[n,p] = B[n,p] + ((p^2 - Nomax^2)/(4 * (Nomax - 1))) * B[n,Nomax-2]
    end
end

# copy MATRIX to MATRIXcal
Acal[1:Ncal-4,1:Ncal-4] .= A[0:N-4,0:N-4]
Bcal[1:Ncal-4,1:Ncal-4] .= B[0:N-4,0:N-4]
A_J[1:Ncal-4,1:Ncal-4] = transpose(Acal[1:Ncal-4,1:Ncal-4])  # A_J = Transpose type
B_J[1:Ncal-4,1:Ncal-4] = transpose(Bcal[1:Ncal-4,1:Ncal-4])  # B_J = Transpose type

Binv = inv(B_J[1:Ncal-4,1:Ncal-4])
Ccal = inv(B_J[1:Ncal-4,1:Ncal-4]) * A_J[1:Ncal-4,1:Ncal-4]
Dcal = eigvals(Ccal)
F = imag.(Dcal)
for i in 1:N-4
    println("Eig: ", Dcal[i])
end

# For Method 2, a_n (N-3<=n<=N) from boundary conditions
## [A-λB]x = 0 -> Ax=λBx にして A, B を QZ 分解 (一般化 Schur 分解) して上三角行列から固有値を求める
for p in 0:2:N  # even mode
    A2[N-3,p] = A2[N-3,p] + 1.0
    A2[N-1,p] = A2[N-1,p] + p^2
end
for p in 1:2:N  # odd mode
    A2[N-2,p] = A2[N-2,p] + 1.0
    A2[N,p] = A2[N,p] + p^2
end

# copy MATRIX to MATRIXcal
A2cal[1:Ncal,1:Ncal] .= A2[0:N,0:N]
B2cal[1:Ncal,1:Ncal] .= B2[0:N,0:N]
# Convert Transpose type to Matrix (Array) type (ドット演算して要素だけコピーしている)
A2_J .= transpose(A2cal)  # A_J = Transpose type
B2_J .= transpose(B2cal)  # B_J = Transpose type

F = schur(A2_J,B2_J)
D2cal = F.α ./ F.β
for i in 1:Ncal
    println("Schur(Eig): ", D2cal[i])
end

#G = eigvecs(Ccal)
#H = inv(G) * Ccal * G
#ncmax = findmax(F)[1]
#Ae = OffsetArray(A[0:div(N,2),0:div(N,2)],0:div(N,2),0:div(N,2))
#Ao = OffsetArray(A[1:div(N,2),1:div(N,2)],1:div(N,2),1:div(N,2))
#Be = OffsetArray(B[0:div(N,2),0:div(N,2)],0:div(N,2),0:div(N,2))
#Bo = OffsetArray(B[1:div(N,2),1:div(N,2)],1:div(N,2),1:div(N,2))
#for n in 0:div(N,2)
#    for p in 0:div(N,2)
#        Ae[n,p] = A[2*n,2*p]
#        Be[n,p] = B[2*n,2*p]
#    end
#end
#for n in 1:div(N,2)
#    for p in 1:div(N,2)
#        Ao[n,p] = A[2*n-1,2*p-1]
#        Bo[n,p] = B[2*n-1,2*p-1]
#    end
#end
#Acal[1:div(N,2)-1,1:div(N,2)-1] .= Ae[0:div(N,2)-2,0:div(N,2)-2]
#Bcal[1:div(N,2)-1,1:div(N,2)-1] .= Be[0:div(N,2)-2,0:div(N,2)-2]
#Ccal = inv(Bcal[1:div(N,2)-1,1:div(N,2)-1]) * Acal[1:div(N,2)-1,1:div(N,2)-1]
##Ccal = inv(Bcal) * Acal
#Dcal = eigvals(Ccal)
#F = imag.(Dcal)
#for i in 1:div(N,2)-1
#    println(F[i])
#end

#Acal[1:div(N,2)-2,1:div(N,2)-2] .= Ao[1:div(N,2)-2,1:div(N,2)-2]
#Bcal[1:div(N,2)-2,1:div(N,2)-2] .= Bo[1:div(N,2)-2,1:div(N,2)-2]
#Ccal = inv(Bcal[1:div(N,2)-2,1:div(N,2)-2]) * Acal[1:div(N,2)-2,1:div(N,2)-2]
##Ccal = inv(Bcal) * Acal
#Dcal = eigvals(Ccal)
#F = imag.(Dcal)
#for i in 1:div(N,2)-2
#    println(F[i])
#end


Nemax = 56, Nomax = 55
Eig: 0.4999999999334932 + 903.0589774662305im
Eig: 0.4999999999988134 - 34.897806452343005im
Eig: 0.4999999999996687 - 0.06311310408499561im
Eig: 0.49999999999970246 - 6.56472786547905im
Eig: 0.49999999999977174 - 0.01192747960575369im
Eig: 0.4999999999998186 - 0.14206996446886708im
Eig: 0.4999999999998308 - 2.684892171972676im
Eig: 0.4999999999998447 - 0.49079903717010587im
Eig: 0.4999999999998891 - 0.029692874205562946im
Eig: 0.49999999999989275 - 0.9336504164573606im
Eig: 0.49999999999990624 - 0.002057057279983828im
Eig: 0.4999999999999148 - 0.019823234068627366im
Eig: 0.4999999999999202 - 0.08891054855459826im
Eig: 0.4999999999999247 - 0.2723463450930247im
Eig: 0.49999999999992484 - 0.2073607808223159im
Eig: 0.4999999999999384 - 1.4669377227922586im
Eig: 0.4999999999999414 - 0.6543836711906931im
Eig: 0.4999999999999503 - 0.055353878991148485im
Eig: 0.49999999999995354 - 0.2373475309261251im
Eig: 0.49999999999995787 - 0.1797111270289892im
Eig: 0.49999999999996

In [2]:
#-- Drawing
##########
#  Plot  #
##########
using PyPlot

rc("font", family="IPAPGothic")
fig = figure("pyplot_majorminor",figsize=(7,5))

x = real.(Dcal[3:Ncal-4])
y = imag.(Dcal[3:Ncal-4])

ax = gca()

cp = scatter(x,y)

ax = gca()

xlabel("X")
ylabel("Y")
#if i == 1 #div(ne,2)-1
#   plt.colorbar(cp)
#end
grid("on")

fig.canvas.draw()
gcf() # Needed for IJulia to plot inline

#PyPlot.title("Re = Re[i]")

fname = "test.png"
#savefig("B-init_2d.pdf")
savefig(fname)
#psi


UndefVarError: [91mUndefVarError: Ncal not defined[39m

In [66]:
#-- Drawing
##########
#  Plot  #
##########
#using PyPlot

#rc("font", family="IPAPGothic")
#fig = figure("pyplot_majorminor",figsize=(7,5))

nx = Ncal-4
ny = Ncal-4

xax = reshape(zeros(nx,ny),nx,ny)
yax = reshape(zeros(nx,ny),nx,ny)

for i in 1:nx
    xax[i,1:ny] .= i  # xmin + (xmax-xmin) * (i-1) / (nx-1)
end
for i in 1:ny
    yax[1:nx,i] .= i  # ymin + (ymax-ymin) * (i-1) / (ny-1)
#    yax[1:nx,i] .= Re[i]  # ymin + (ymax-ymin) * (i-1) / (ny-1)
end

cont_val = reshape(zeros(nx,ny),nx,ny)
shade_val = reshape(zeros(nx,ny),nx,ny)
#for i in 1:nr#div(nr,2)-1:div(nr,2)+1
    cont_val = imag.(Ccal[1:nx,1:ny])
    #for j in 1:ne
    shade_val = imag.(A_J[1:nx,1:ny])
    #end
#draw_num2 = 2
#if draw_num2 == 1
#    cp = contourf(xax[1:nx,1:nx], xax'[1:nx,1:nx], Pftime[1:nx,1:nx,1], levels=[-15.0, -10.0, -5.0, 0.0, 5.0, 10.0, 15.0, 20.0], origin="image", cmap=ColorMap("viridis"), extend="both")
    #cp = contourf(xax[1:nx,1:nx], xax'[1:nx,1:nx], Pftime[1:nx,1:nx,2], levels=[-2.0, -1.0, -0.5, -0.25, 0.25, 0.5, 1.0, 1.5, 2.0], origin="image", cmap=ColorMap("viridis"), extend="both")
    #cp = contourf(xax[1:nx,1:no], xax'[1:nx,1:no], Pftime[1:nx,1:nx,2]*Hop'*inv(Hop*Pftime[1:nx,1:nx,2]*Hop'+Ro), origin="image", cmap=ColorMap("viridis"), extend="both")
#elseif draw_num2 == 2
    ##cp = contour(xax[1:nx,1:ny], yax[1:nx,1:ny], shade_val[1:nx,1:ny], 8, levels=[-0.01, -0.005, -0.003, -0.002, -0.001, -0.0005, 0.0, 0.0005, 0.001, 0.002, 0.003, 0.005, 0.01])
    cp = contourf(xax[1:nx,1:ny], yax[1:nx,1:ny], cont_val[1:nx,1:ny], cmap=ColorMap("viridis"), extend="both")
    ##cp = contourf(xax[1:nx,1:ny], yax[1:nx,1:ny], cont_val[1:nx,1:ny], 8, levels=[-1.0, -0.5, -0.2, -0.1, 0.0, 0.1, 0.2, 0.5, 1.0], cmap=ColorMap("viridis"), extend="both")
    #cp = contourf(xax[1:nx,1:ny], yax[1:nx,1:ny], shade_val[1:nx,1:ny], 10, cmap=ColorMap("viridis"), extend="both")
#elseif draw_num2 == 3
#    cp = contourf(xoax[1:nx,1:nto-1], toax[1:nx,1:nto-1], evec_max[1:nx,1:nto-1], 10, cmap=ColorMap("viridis"), extend="both")
#elseif draw_num2 == 4
#    cp = contourf(xoax[1:nx,1:40], toax[1:nx,1:40], x_inc[1:nx,1:40], levels=[-2.0, -1.0, -0.5, -0.25, 0.25, 0.5, 1.0, 1.5, 2.0], cmap=ColorMap("viridis"), extend="both")
#elseif draw_num2 == 5
#    cp = contourf(xoax[1:no,1:40], toax[1:no,1:40], y_innov[1:no,1:40], levels=[-2.0, -1.0, -0.5, -0.25, 0.25, 0.5, 1.0, 1.5, 2.0], cmap=ColorMap("viridis"), extend="both")
#end
#ax.label(cp, inline=1, fontsize=10)
#legend()
ax = gca()

xlabel("X")
ylabel("Y")
#if i == 1 #div(ne,2)-1
   plt.colorbar(cp)
#end
grid("on")

PyPlot.title("Re = Re[i]")

#########################
#  Set tick dimensions  #
#########################
#ax.xaxis.set_tick_params(which="major",length=5,width=2,labelsize=10)
#ax.xaxis.set_tick_params(which="minor",length=5,width=2)

fig.canvas.draw() # Update the figure
gcf() # Needed for IJulia to plot inline
#fname = "Re" * string(1) * ".png"
fname = "test.png"
#savefig("B-init_2d.pdf")
savefig(fname)
#psi
#end
#println(shade_val[1:na,1])

Figure(PyObject <Figure size 640x480 with 2 Axes>)

In [47]:
a33 = reshape(zeros(Complex{Float64},2,2),2,2)
c33 = reshape(zeros(Complex{Float64},2,2),2,2)
println(a33)
a33[1,1] = 1.0 + 2.0im
a33[2,2] = 2.0 - 1.0im
b33 = transpose(a33)
c33 .= b33
schur(a33,c33)

ComplexF64[0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im]


GeneralizedSchur{ComplexF64, Matrix{ComplexF64}}
S factor:
2×2 Matrix{ComplexF64}:
 2.23607-0.0im      0.0+0.0im
     0.0+0.0im  2.23607+0.0im
T factor:
2×2 Matrix{ComplexF64}:
 2.23607+0.0im      0.0+0.0im
     0.0+0.0im  2.23607+0.0im
Q factor:
2×2 Matrix{ComplexF64}:
 -0.447214-0.894427im  0.0+0.0im
       0.0+0.0im       1.0+0.0im
Z factor:
2×2 Matrix{ComplexF64}:
 -1.0-0.0im       0.0+0.0im
 -0.0-0.0im  0.894427+0.447214im
α:
2-element Vector{ComplexF64}:
 2.23606797749979 - 0.0im
 2.23606797749979 + 0.0im
β:
2-element Vector{ComplexF64}:
 2.23606797749979 + 0.0im
 2.23606797749979 + 0.0im